# **Agents**
1. Function calling agent
2. ReAct Agent
3. Advanced Custom Agent

In [1]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.core.tools import FunctionTool

def multiply(a: int, b: int) ->int:
    """
    Multiplies two integers and returns the resulting integer.
    """
    return a * b
llm = HuggingFaceInferenceAPI(model_name = "Qwen/Qwen2.5-Coder-32B-Instruct")

agent = AgentWorkflow.from_tools_or_functions(
    [FunctionTool.from_defaults(multiply)],
    llm=llm
)

* Agents are stateless by defaults.
* We add past interactions using a Context object.

In [2]:
response = await agent.run("What is 2 times 2")
response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='2 times 2 is 4.')]), tool_calls=[ToolCallResult(tool_name='multiply', tool_kwargs={'a': 2, 'b': 2}, tool_id='20f2e5db-84de-4523-a0cc-15000825a7bc', tool_output=ToolOutput(content='4', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 2, 'b': 2}}, raw_output=4, is_error=False), return_direct=False)], raw=ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role='assistant', content='.', tool_calls=None), index=0, finish_reason=None, logprobs=None)], created=1741456989, id='', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint='3.0.1-sha-bb9095a', usage=None), current_agent_name='Agent')

In [3]:
from llama_index.core.workflow import Context

ctx = Context(agent)
response = await agent.run("My name is Bob.",ctx=ctx)
response = await agent.run("What was my name again?",ctx=ctx)
response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Your name is Bob.')]), tool_calls=[ToolCallResult(tool_name='multiply', tool_kwargs={'a': 3, 'b': 4}, tool_id='02002503-00d0-41ef-a1e9-2bc256ef9c09', tool_output=ToolOutput(content='12', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 3, 'b': 4}}, raw_output=12, is_error=False), return_direct=False)], raw=ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role='assistant', content='.', tool_calls=None), index=0, finish_reason=None, logprobs=None)], created=1741457090, id='', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint='3.0.1-sha-bb9095a', usage=None), current_agent_name='Agent')

* The AgentWorkflow class also directly supports multi-agent systems. By giving each agent a name and description, the system maintains a single active speaker, with each agent having the ability to hand off to another agent.

* By narrowing the scope of each agent, we can help increase their general accuracy when responding to user messages.

* Agents in LlamaIndex can also directly be used as tools for other agents, for more complex and custom scenarios

# **Multi Agent system**

In [4]:
from llama_index.core.agent.workflow import (
    AgentWorkflow,
    FunctionAgent,ReActAgent
)

* Function agent works with function calling llm
* ReAct Agent works with any llm.

In [7]:
def add(a:int, b:int)-> int:
    """
    Add two numbers.
    """
    return a + b

def subtract(a:int,b:int)->int:
    """
    Subtract two numbers.
    """
    return a - b

calculator_agent = ReActAgent(
    name="calculator",
    description="Performs basic arithmetic operations",
    system_prompt="You are a calculator assistant.Use your tools for any math operations.",
    tools=[add,subtract],
    llm=llm
)

In [ ]:
query_agent = ReActAgent(
    name = "info_lookup",
    description="Look up information about XYZ",
    system_prompt="Use your tool to query a RAG system to answer information about XYZ",
    tools=[query_engine_tool],
    llm=llm
)

agent = AgentWorkflow(
    agents=[calculator_agent,query_agent], root_agent="calculator"
)

response = await agent.run(user_msg="Can you add 5 and 3?")